In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

Load the dataset

In [ ]:
iris=datasets.load_iris()


Create a dataframe by defining sepal length and width in x and y respectively. The type of flower is 0, 1 or 2 which comes from iris.target in "cluster"

In [ ]:
df=pd.DataFrame({'x': iris.data[:,0], 'y': iris.data[:,1],'cluster':iris.target})

In [ ]:
df

Here we are interested to find 3 clusters so k=3, hence first three centrods are created using "Mean" of each x value for same kind of flower. Similarily, mean y is found for all three types of flowers.
These x and y are resulting into 3 centroids.
Initializing of centroids

In [ ]:
centroids={}
for i in range(3):
  result_list=[]
  result_list.append(df.loc[df['cluster']==i]['x'].mean())
  result_list.append(df.loc[df['cluster']==i]['y'].mean())
  
  centroids[i]=result_list


Centroids and there plotting on graph

In [ ]:
centroids

In [ ]:
fig=plt.figure(figsize=(5,5))
plt.scatter(df['x'], df['y'],c=iris.target, alpha=0.3)
#plt.xlabel("Sepal length", fontsize=18)
#plt.ylabel("Sepal width", fontsize=18)

colmap={0:'r', 1:'g',2:'b'}
col=[0,1]

for i in centroids.keys():
  plt.scatter(centroids[i][0],centroids[i][1], color=colmap[i], edgecolor='k')
plt.show()


A method assignment is created to measure, given a point whether it is similar to centrod 0, 1 or 2, based on which that point will be allocated to a cluster.
Here, Euclidean distance measure is used to find similarity between point and centroid.

In [ ]:
def assignment(df, centroids):
  for i in range(3):
    df['distance_from_{}'.format(i)]=(np.sqrt((df['x']-centroids[i][0])**2+(df['y']-centroids[i][1])**2))
  centroid_distance_cols=['distance_from_{}'.format(i) for i in centroids.keys() ]
  df['closest']=df.loc[:,centroid_distance_cols].idxmin(axis=1)
  df['closest']= df['closest'].map(lambda x :int (x.lstrip('distance_from_')))
  df['color']= df['closest'].map(lambda x: colmap[x])
  return df

In [ ]:
df=assignment(df,centroids)



Using the assignment(), distance of point from each cluster is found and the data frame(which initially had 3 features, now has 5 more features) has in all 8 features. Based on the value of "distance_from_0", "distance_from_1" and "distance_from_2", the point is found closest to any one of the centroid and accordingly flower type is finalized.

In [ ]:
df

In [ ]:
fig=plt.figure(figsize=(5,5))
plt.scatter(df['x'], df['y'],color=df['color'], alpha=0.3)


for i in centroids.keys():
  plt.scatter(*centroids[i], color=colmap[i], edgecolor='k')
plt.show()


It may happen that as soon as the points are allocated, the mean of each cluster may be affected. Hence update() function is created to locate new centroids.

In [ ]:
def update(k):
  for i in range(3):
    centroids[i][0]=np.mean(df[df['closest']==i]['x'])
    centroids[i][1]=np.mean(df[df['closest']==i]['y'])
  return k

In [ ]:
centroids=update(centroids)
centroids

After updating centroids, again the euclidean distance between each point and new centroid is to be calculated. This iterative process stops when no change/update in centroids is found.

In [ ]:
while True:
  closest_centroids=df['closest'].copy(deep=True)
  centroids=update(centroids)
  df=assignment(df,centroids)
  if closest_centroids.equals(df['closest']):
    break;


Plotting of new centroids

In [ ]:
fig=plt.figure(figsize=(5,5))
plt.scatter(df['x'], df['y'],color=df['color'], alpha=0.3)


for i in centroids.keys():
  plt.scatter(*centroids[i], color=colmap[i], edgecolor='k')
plt.show()


Instead of writing our own assignment() and update() and their related iterations, just use KMeans of sklearn.

In [ ]:
from sklearn.cluster import KMeans
kmeans=KMeans(n_clusters=3)
dfnew=pd.DataFrame({'x': iris.data[:,0], 'y': iris.data[:,1]})
kmeans.fit(dfnew)


In [ ]:
labels = kmeans.predict(dfnew)
newcentroids = kmeans.cluster_centers_

In [ ]:
newcentroids
